## Creation of  a Counting Model with Background uncertainty

In this notebook we extend the previous Poisson counting model by considering $b$ not fixed but varying according to a given distribution. We assume we know b from a previous experiment, i.e. we have measured $b$ to have a value $b_0$ with an uncertainty $\sigma_b$. 

The model describing the experiment is given by a Poisson distribution multiplied by a Gaussian distribution for the background  

$$ P(n_{obs} | s,b) = Poisson (n_{obs} | s+b) \times Gaussian(b_0 | b, \sigma_b) = (s+b)^{n_{obs} } \frac{ e^{-(s+b)} } { n_{obs} ! } \times \frac{1}{\sqrt{2 \pi \sigma_b^2 } } e^{ (b_0-b)^2 / \sigma_b^2}$$


We will create the model using **RooFit** and **RooStats**. 
See the links below for more information
-  RooFit: https://cernbox.cern.ch/index.php/s/kzGv4406IrpFpVM
-  RooStats: https://cernbox.cern.ch/index.php/s/9DWJCObingTWB3r

In [ ]:
int nobs; double b; double sigmab;

In [ ]:
nobs = 4;                // number of observed events
b = 3;                 // number of expected background even
sigmab = 0.3;          // uncertainty on b

#### Model Creation using `RooWorkspace` 

Using the factory syntax of the `RooWorkspace` we create the parameters of the model and the Poisson p.d.f describing the model.

In [ ]:
RooWorkspace w("w");
   
// make Poisson model * Gaussian constraint
w.factory("sum:nexp(s[3,0,15],b[1,0,10])");
// Poisson of (n | s+b)
w.factory("Poisson:pdf(nobs[0,50],nexp)");
// Gaussian for b
w.factory("Gaussian:constraint(b0[0,10],b,sigmab[1])");
// Full model
w.factory("PROD:model(pdf,constraint)");

#### Creation of  the  `RooStats::ModelConfig` class

We create the `ModelConfig` class, which is needed to run all `RooStats` statistics calculators. 
The class contains meta-information on the model, such as the name of the pdf, the observables, the parameter of interest and if existing the nuisance parameters. 

In [ ]:
RooStats::ModelConfig mc("ModelConfig",&w);
mc.SetPdf(*w.pdf("model"));
mc.SetParametersOfInterest(*w.var("s"));
mc.SetObservables(*w.var("nobs"));
// we set here the nuisance parameters 
mc.SetNuisanceParameters(*w.var("b"));
// this is needed for dealing of the systematics in b
// we assume we have an  additional observables b0
mc.SetGlobalObservables(*w.var("b0"));


// this is needed for the hypothesis tests
mc.SetSnapshot(*w.var("s"));

mc.Print();
// import model in the workspace 
w.import(mc);

#####  Setting  the model parameter values  ( e.g. number of background events )

We set the assumed value of b, and  since we consider it fixed (known precisly) it is important to set it constant.  

In [ ]:
w.var("b")->setVal(b); 
w.var("b0")->setVal(b);
w.var("b0")->setConstant(true); // needed for being treated as global observables
w.var("sigmab")->setVal(sigmab*b);    

#### Data Set generation

We create here the observed data set for the model. The data set will conist of a single event with the observable 
$n_{obs}$ equal to the given value (e.g. 3). 

It is important to import the data set in the workspace


In [ ]:
RooDataSet data("data","", *w.var("nobs"));
w.var("nobs")->setVal(nobs);
data.add(*w.var("nobs") );
// import data set in workspace 
w.import(data);
std::cout << "number of observed events  = " << data.get(0)->getRealValue("nobs") << std::endl;

#### Save Workspace in a file

We save the generated model as a workspace in a file that can be used later on. 
Before saving we print its content for showing its content. 

In [ ]:
w.Print();
// print also model parameters 
std::cout << "Parameter of interest:  \n"; mc.GetParametersOfInterest()->Print("v");
std::cout << "Nuisance parameters  :  \n"; mc.GetNuisanceParameters()->Print("v");
w.writeToFile("CountingModelVaryingB.root", true);
